In [ ]:
import polars as pl  # noqa: PLC0415


In [ ]:
input_dir = "/home/vandy/Work/MATH6310/blunder-analysis/data/silver"

In [ ]:
df = pl.scan_parquet(f"{input_dir}/*.parquet")
# Check for duplicate offsets
offset_counts = (df
    .group_by(["offset", "fullmove_number", "turn"])
    .agg(pl.count().alias("count"))
    .filter(pl.col("count") > 1)
    .collect()
)
offset_counts